In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

_join = os.path.join

### Summary:

The purpose of this jupyter notebook is to summarize the benefits for super districts and for the entire region. The output csv files from this process is moved to the excel spreadsheet for creating regional summaries. 
 

In [2]:
purpose = ['Work', 'University', 'School', 'Escort', 'Shopping', 'EatOut', 
           'OthMaint', 'Social', 'OthDiscr', 'WorkBased']

time_period = {1:'EA',2:'AM',3:'MD',4:'PM',5:'EV'} #1 for EA, 2 for AM, 3 for MD, 4 for PM and 5 for EV


auto_suff_cat = ['autoSufficient', 'autoDeficient', 'zeroAuto']

In [3]:
# model_folders = ['W:\TM2_2050R39_R2_Run4', 'W:\TM2_2050R41_R2_Run4', 'W:\TM2_2050R40_R2_Run2']
# model_dir = 'W:\TM2_2050STR39_R2_Run2' #update this to run for R39/R41/R40
# model_dir = r'W:\TM2_2050STR40_R2_Run1'
# model_dir = r'W:\TM2_2050R39_R2_Run6'
# model_dir = r'W:\TM2_2050STR40_R2_Run1_VY'
# model_dir = r'W:\TM2_2050Baseline_R2_Run7'
# model_dir = r'W:\TM2_2050STR40_R2_Run1_VY'
# model_dir = r'D:\TM2_2050R40_R2_Run2_Conv'
# model_dir = r'Z:\TM2_2050_BL_R2_Run6_TollTest_Run2'
# model_dir = r'Z:\TM2_2050_BL_R2_Run6_COVID_Run3'
# model_dir = r'Z:\TM2_2050_R39_R2_Run6_Cnvrg'
model_dir = r'Z:\TM2_2050_STR39_Run2_Cnvrg'


benefits_folder = _join(model_dir, 'benefits13')

geo_cwks = pd.read_csv(_join("geographies.csv")) #columns taz, rdm_zones, super_district, county
geo_cwks_dict = dict(zip(geo_cwks['taz'], geo_cwks['super_district']))

In [4]:
def create_sd_summary(benefits_folder,
                      column_ext, 
                      time_period, 
                      auto_suff_cat, 
                      purpose):
    
    """creates super district to super district summaries """

    # Loop through time periods and purposes
    sd_summary_all = []
    for tripPeriod in time_period:
        tp_name = time_period[tripPeriod]

        for auto_suff in auto_suff_cat:

            for purp in purpose:

                df = pd.read_parquet(_join(benefits_folder, f"{column_ext}_{tp_name}_{purp}_{auto_suff}.parquet"))

                df['orig_sd'] = df['orig_taz'].map(geo_cwks_dict)
                df['dest_sd'] = df['dest_taz'].map(geo_cwks_dict)
                df['time_period'] = tp_name
                df['purpose'] = purp
                df['auto_suff'] = auto_suff

                sd_summary = df.groupby(['orig_sd', 'dest_sd', 'time_period', 'purpose', 'auto_suff'])['auto_'+column_ext, 
                                                                          'transit_'+column_ext,
                                                                          'non-motorized_'+column_ext,
                                                                          'ridehail_'+column_ext].sum().reset_index()
                sd_summary_all.append(sd_summary)
                
    print("writing summary file")
    sd_summary_all = pd.concat(sd_summary_all)
    sd_summary_all.to_csv(_join(benefits_folder, f"sd_summary_{column_ext}.csv"), index=False)

In [5]:
%%time

# benefits by proportion method  (Method 1)
#create_sd_summary(benefits_folder, 'benefits_pr', time_period, auto_suff_cat, purpose)

# existing user benefits (Method 1)
#create_sd_summary(benefits_folder, 'benefits_nu', time_period, auto_suff_cat, purpose)

# new user benefits (Method 1)
#create_sd_summary(benefits_folder, 'benefits_eu', time_period, auto_suff_cat, purpose)

Wall time: 0 ns


In [6]:
%%time

#benefits by logsum method (Method 2)
#create_sd_summary(benefits_folder, 'benefits_ls', time_period, auto_suff_cat, purpose)

# existing user benefits (Method 2)
#create_sd_summary(benefits_folder, 'benefits_nu_ls', time_period, auto_suff_cat, purpose)

# new user benefits (Method 2)
#create_sd_summary(benefits_folder, 'benefits_eu_ls', time_period, auto_suff_cat, purpose)

Wall time: 0 ns


In [5]:
%%time

# benefits by logsum method (Method 3)
create_sd_summary(benefits_folder, 'benefits_cls', time_period, auto_suff_cat, purpose)

# existing user benefits (Method 3)
# create_sd_summary(benefits_folder, 'benefits_nu_cls', time_period, auto_suff_cat, purpose)

# new user benefits (Method 3)
# create_sd_summary(benefits_folder, 'benefits_eu_cls', time_period, auto_suff_cat, purpose)

writing summary file
Wall time: 12min 31s


In [6]:
%%time

# create sd to sd trips
trips_folder = _join(model_dir, 'trips')
create_sd_summary(trips_folder, 'trips', time_period, auto_suff_cat, purpose)

writing summary file
Wall time: 12min 32s


#### Summarize all benefits by excluding/including intrazonal benefits

%%time

# Create an empty list to store the data
data_incintz = []
data_excintz = []

summary_columns = ['benefits_pr', # benefits by proportion method - Method 1
                  'benefits_ls', # benefits by logsum method - Method 2
                  'benefits_cls', # benefits by composite logsum method - Method 3
                   ]

for column_ext in summary_columns:

    # Loop through time periods and purposes
    for tripPeriod in time_period:
        tp_name = time_period[tripPeriod]

        for auto_suff in auto_suff_cat:

            for purp in purpose:

                purp_benefit = omx.open_file(_join(benefits_folder, f"benefits_{tp_name}_{purp}_{auto_suff}_bivt_mp.omx"))

                auto_benefits = np.array(purp_benefit['auto_' + column_ext])
                trn_benefits = np.array(purp_benefit['trn_' + column_ext])
                nm_benefits = np.array(purp_benefit['nm_' + column_ext])
                rh_benefits = np.array(purp_benefit['rh_' + column_ext])

                purp_benefit.close()

                # Calculate the sums
                auto_sum = np.sum(auto_benefits)
                trn_sum = np.sum(trn_benefits)
                nm_sum = np.sum(nm_benefits)
                rh_sum = np.sum(rh_benefits)

                # Append the data to the list
                data_incintz.append([purp, tp_name, auto_suff, auto_sum, trn_sum, nm_sum, rh_sum])

                #fill diagonals with zero
                np.fill_diagonal(auto_benefits, 0)
                np.fill_diagonal(trn_benefits, 0)
                np.fill_diagonal(nm_benefits, 0)
                np.fill_diagonal(rh_benefits, 0)

                # Calculate the sums
                auto_sum = np.sum(auto_benefits)
                trn_sum = np.sum(trn_benefits)
                nm_sum = np.sum(nm_benefits)
                rh_sum = np.sum(rh_benefits)

                # Append the data to the list
                data_excintz.append([purp, tp_name, auto_suff, auto_sum, trn_sum, nm_sum, rh_sum])

    # Create a DataFrame from the list
    inc_intz = pd.DataFrame(data_incintz, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
    exc_intz = pd.DataFrame(data_excintz, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])

    # Save the DataFrame to a CSV file
    inc_intz.to_csv(_join(benefits_folder, column_ext + '_benefits_data_incintz_bivt.csv'), index=False)
    exc_intz.to_csv(_join(benefits_folder, column_ext +'_benefits_data_excintz_bivt.csv'), index=False)

In [ ]:
### Other summaries

In [7]:
%%time

#benefits_folder = r"W:\TM2_2050R41_R2_Run4\benefits13"
#benefits_folder = r'W:\TM2_2050R39_R2_Run4\benefits13'
#benefits_folder = r'W:\TM2_2050R40_R2_Run2\benefits13'
#benefits_folder = r'W:\TM2_2050STR39_R2_Run2\benefits13'
#benefits_folder = r'W:\TM2_2050STR40_R2_Run1\benefits13'
#benefits_folder = r'W:\TM2_2050STR41_R2_Run2\benefits13'

# Create an empty list to store the data
data_excintz_ls = []
data_excintz_nu_ls = []
data_excintz_eu_ls = []
data_excintz_cls = []
data_excintz_nu_cls = []
data_excintz_eu_cls = []

# Loop through time periods and purposes
for tripPeriod in time_period:
    tp_name = time_period[tripPeriod]
    print(tp_name)
    
    for auto_suff in auto_suff_cat:
        
        for purp in purpose:
        
            purp_benefit = omx.open_file(_join(benefits_folder, f"benefits_{tp_name}_{purp}_{auto_suff}_bivt_mp.omx"))
                        
            #all benefits using logsums
            auto_benefits_ls = np.array(purp_benefit['auto_benefits_ls'])
            trn_benefits_ls = np.array(purp_benefit['trn_benefits_ls'])
            nm_benefits_ls = np.array(purp_benefit['nm_benefits_ls'])
            rh_benefits_ls = np.array(purp_benefit['rh_benefits_ls'])
            
            #all benefits using logsums
            auto_benefits_cls = np.array(purp_benefit['auto_benefits_cls'])
            trn_benefits_cls = np.array(purp_benefit['trn_benefits_cls'])
            nm_benefits_cls = np.array(purp_benefit['nm_benefits_cls'])
            rh_benefits_cls = np.array(purp_benefit['rh_benefits_cls'])
            
            #all benefits using proportion for new users
            auto_benefits_nu_ls = np.array(purp_benefit['auto_benefits_nu_ls'])
            trn_benefits_nu_ls = np.array(purp_benefit['trn_benefits_nu_ls'])
            nm_benefits_nu_ls = np.array(purp_benefit['nm_benefits_nu_ls'])
            rh_benefits_nu_ls = np.array(purp_benefit['rh_benefits_nu_ls'])
            
            #all benefits using proportion for existing users
            auto_benefits_eu_ls = np.array(purp_benefit['auto_benefits_eu_ls'])
            trn_benefits_eu_ls = np.array(purp_benefit['trn_benefits_eu_ls'])
            nm_benefits_eu_ls = np.array(purp_benefit['nm_benefits_eu_ls'])
            rh_benefits_eu_ls = np.array(purp_benefit['rh_benefits_eu_ls'])
            
            #all benefits using proportion for new users
            auto_benefits_nu_cls = np.array(purp_benefit['auto_benefits_nu_cls'])
            trn_benefits_nu_cls = np.array(purp_benefit['trn_benefits_nu_cls'])
            nm_benefits_nu_cls = np.array(purp_benefit['nm_benefits_nu_cls'])
            rh_benefits_nu_cls = np.array(purp_benefit['rh_benefits_nu_cls'])
            
            #all benefits using proportion for existing users
            auto_benefits_eu_cls = np.array(purp_benefit['auto_benefits_eu_cls'])
            trn_benefits_eu_cls = np.array(purp_benefit['trn_benefits_eu_cls'])
            nm_benefits_eu_cls = np.array(purp_benefit['nm_benefits_eu_cls'])
            rh_benefits_eu_cls = np.array(purp_benefit['rh_benefits_eu_cls'])
                        
            purp_benefit.close()
            
            
            np.fill_diagonal(auto_benefits_ls, 0)
            np.fill_diagonal(trn_benefits_ls, 0)
            np.fill_diagonal(nm_benefits_ls, 0)
            np.fill_diagonal(rh_benefits_ls, 0)
            
            np.fill_diagonal(auto_benefits_cls, 0)
            np.fill_diagonal(trn_benefits_cls, 0)
            np.fill_diagonal(nm_benefits_cls, 0)
            np.fill_diagonal(rh_benefits_cls, 0)
            
            np.fill_diagonal(auto_benefits_nu_ls, 0)
            np.fill_diagonal(trn_benefits_nu_ls, 0)
            np.fill_diagonal(nm_benefits_nu_ls, 0)
            np.fill_diagonal(rh_benefits_nu_ls, 0)
            
            np.fill_diagonal(auto_benefits_eu_ls, 0)
            np.fill_diagonal(trn_benefits_eu_ls, 0)
            np.fill_diagonal(nm_benefits_eu_ls, 0)
            np.fill_diagonal(rh_benefits_eu_ls, 0)
            
            np.fill_diagonal(auto_benefits_nu_cls, 0)
            np.fill_diagonal(trn_benefits_nu_cls, 0)
            np.fill_diagonal(nm_benefits_nu_cls, 0)
            np.fill_diagonal(rh_benefits_nu_cls, 0)
            
            np.fill_diagonal(auto_benefits_eu_cls, 0)
            np.fill_diagonal(trn_benefits_eu_cls, 0)
            np.fill_diagonal(nm_benefits_eu_cls, 0)
            np.fill_diagonal(rh_benefits_eu_cls, 0)
            
                        
            
            # Calculate the sums
            auto_sum_ls = np.sum(auto_benefits_ls)
            trn_sum_ls = np.sum(trn_benefits_ls)
            nm_sum_ls = np.sum(nm_benefits_ls)
            rh_sum_ls = np.sum(rh_benefits_ls)
            
            # Calculate the sums
            auto_sum_cls = np.sum(auto_benefits_cls)
            trn_sum_cls = np.sum(trn_benefits_cls)
            nm_sum_cls = np.sum(nm_benefits_cls)
            rh_sum_cls = np.sum(rh_benefits_cls)
            
            # Calculate the sums
            auto_sum_nu_ls = np.sum(auto_benefits_nu_ls)
            trn_sum_nu_ls= np.sum(trn_benefits_nu_ls)
            nm_sum_nu_ls = np.sum(nm_benefits_nu_ls)
            rh_sum_nu_ls = np.sum(rh_benefits_nu_ls)
            
            # Calculate the sums
            auto_sum_eu_ls = np.sum(auto_benefits_eu_ls)
            trn_sum_eu_ls = np.sum(trn_benefits_eu_ls)
            nm_sum_eu_ls = np.sum(nm_benefits_eu_ls)
            rh_sum_eu_ls = np.sum(rh_benefits_eu_ls)
            
            # Calculate the sums
            auto_sum_nu_cls = np.sum(auto_benefits_nu_cls)
            trn_sum_nu_cls = np.sum(trn_benefits_nu_cls)
            nm_sum_nu_cls = np.sum(nm_benefits_nu_cls)
            rh_sum_nu_cls = np.sum(rh_benefits_nu_cls)
            
            # Calculate the sums
            auto_sum_eu_cls = np.sum(auto_benefits_eu_cls)
            trn_sum_eu_cls = np.sum(trn_benefits_eu_cls)
            nm_sum_eu_cls = np.sum(nm_benefits_eu_cls)
            rh_sum_eu_cls = np.sum(rh_benefits_eu_cls)
            

            # Append the data to the list
            data_excintz_ls.append([purp, tp_name, auto_suff, auto_sum_ls, trn_sum_ls, nm_sum_ls, rh_sum_ls])
            data_excintz_nu_ls.append([purp, tp_name, auto_suff, auto_sum_nu_ls, trn_sum_nu_ls, nm_sum_nu_ls, rh_sum_nu_ls])
            data_excintz_eu_ls.append([purp, tp_name, auto_suff, auto_sum_eu_ls, trn_sum_eu_ls, nm_sum_eu_ls, rh_sum_eu_ls])
            data_excintz_cls.append([purp, tp_name, auto_suff, auto_sum_cls, trn_sum_cls, nm_sum_cls, rh_sum_cls])
            data_excintz_nu_cls.append([purp, tp_name, auto_suff, auto_sum_nu_cls, trn_sum_nu_cls, nm_sum_nu_cls, rh_sum_nu_cls])
            data_excintz_eu_cls.append([purp, tp_name, auto_suff, auto_sum_eu_cls, trn_sum_eu_cls, nm_sum_eu_cls, rh_sum_eu_cls])
           
                 
# Create a DataFrame from the list
exc_intz_ls = pd.DataFrame(data_excintz_ls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_nu_ls = pd.DataFrame(data_excintz_nu_ls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_eu_ls = pd.DataFrame(data_excintz_eu_ls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_cls = pd.DataFrame(data_excintz_cls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_nu_cls = pd.DataFrame(data_excintz_nu_cls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_eu_cls = pd.DataFrame(data_excintz_eu_cls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])

# Save the DataFrame to a CSV file
exc_intz_ls.to_csv(_join(benefits_folder,'benefits_data_excintz_bivt_ls.csv'), index=False)
exc_intz_nu_ls.to_csv(_join(benefits_folder,'benefits_data_excintz_bivt_nu_ls.csv'), index=False)
exc_intz_eu_ls.to_csv(_join(benefits_folder,'benefits_data_excintz_bivt_eu_ls.csv'), index=False)
exc_intz_cls.to_csv(_join(benefits_folder,'benefits_data_excintz_bivt_cls.csv'), index=False)
exc_intz_nu_cls.to_csv(_join(benefits_folder,'benefits_data_excintz_bivt_nu_cls.csv'), index=False)
exc_intz_eu_cls.to_csv(_join(benefits_folder,'benefits_data_excintz_bivt_eu_cls.csv'), index=False)

EA
AM
MD
PM
EV
Wall time: 14min 46s


%%time

#benefits_folder = r"W:\TM2_2050R41_R2_Run4\benefits13"
#benefits_folder = r'W:\TM2_2050R39_R2_Run4\benefits13'
#benefits_folder = r'W:\TM2_2050R40_R2_Run2\benefits13'
#benefits_folder = r'W:\TM2_2050STR39_R2_Run2\benefits13'
#benefits_folder = r'W:\TM2_2050STR40_R2_Run1\benefits13'
#benefits_folder = r'W:\TM2_2050STR41_R2_Run2\benefits13'

# Create an empty list to store the data
data_excintz_ls = []
data_excintz_nu_ls = []
data_excintz_eu_ls = []
data_excintz_cls = []
data_excintz_nu_cls = []
data_excintz_eu_cls = []

# Loop through time periods and purposes
for tripPeriod in time_period:
    tp_name = time_period[tripPeriod]
    print(tp_name)
    
    for auto_suff in auto_suff_cat:
        
        for purp in purpose:
        
            purp_benefit = omx.open_file(_join(benefits_folder, f"benefits_{tp_name}_{purp}_{auto_suff}_bivt_mp.omx"))
                        
            #all benefits using logsums
            auto_benefits_ls = np.array(purp_benefit['auto_benefits_ls'])
            trn_benefits_ls = np.array(purp_benefit['trn_benefits_ls'])
            nm_benefits_ls = np.array(purp_benefit['nm_benefits_ls'])
            rh_benefits_ls = np.array(purp_benefit['rh_benefits_ls'])
            
            #all benefits using logsums
            auto_benefits_cls = np.array(purp_benefit['auto_benefits_cls'])
            trn_benefits_cls = np.array(purp_benefit['trn_benefits_cls'])
            nm_benefits_cls = np.array(purp_benefit['nm_benefits_cls'])
            rh_benefits_cls = np.array(purp_benefit['rh_benefits_cls'])
            
            #all benefits using proportion for new users
            auto_benefits_nu_ls = np.array(purp_benefit['auto_benefits_nu_ls'])
            trn_benefits_nu_ls = np.array(purp_benefit['trn_benefits_nu_ls'])
            nm_benefits_nu_ls = np.array(purp_benefit['nm_benefits_nu_ls'])
            rh_benefits_nu_ls = np.array(purp_benefit['rh_benefits_nu_ls'])
            
            #all benefits using proportion for existing users
            auto_benefits_eu_ls = np.array(purp_benefit['auto_benefits_eu_ls'])
            trn_benefits_eu_ls = np.array(purp_benefit['trn_benefits_eu_ls'])
            nm_benefits_eu_ls = np.array(purp_benefit['nm_benefits_eu_ls'])
            rh_benefits_eu_ls = np.array(purp_benefit['rh_benefits_eu_ls'])
            
            #all benefits using proportion for new users
            auto_benefits_nu_cls = np.array(purp_benefit['auto_benefits_nu_cls'])
            trn_benefits_nu_cls = np.array(purp_benefit['trn_benefits_nu_cls'])
            nm_benefits_nu_cls = np.array(purp_benefit['nm_benefits_nu_cls'])
            rh_benefits_nu_cls = np.array(purp_benefit['rh_benefits_nu_cls'])
            
            #all benefits using proportion for existing users
            auto_benefits_eu_cls = np.array(purp_benefit['auto_benefits_eu_cls'])
            trn_benefits_eu_cls = np.array(purp_benefit['trn_benefits_eu_cls'])
            nm_benefits_eu_cls = np.array(purp_benefit['nm_benefits_eu_cls'])
            rh_benefits_eu_cls = np.array(purp_benefit['rh_benefits_eu_cls'])
                        
            purp_benefit.close()
            
            """
            np.fill_diagonal(auto_benefits_ls, 0)
            np.fill_diagonal(trn_benefits_ls, 0)
            np.fill_diagonal(nm_benefits_ls, 0)
            np.fill_diagonal(rh_benefits_ls, 0)
            
            np.fill_diagonal(auto_benefits_cls, 0)
            np.fill_diagonal(trn_benefits_cls, 0)
            np.fill_diagonal(nm_benefits_cls, 0)
            np.fill_diagonal(rh_benefits_cls, 0)
            
            np.fill_diagonal(auto_benefits_nu_ls, 0)
            np.fill_diagonal(trn_benefits_nu_ls, 0)
            np.fill_diagonal(nm_benefits_nu_ls, 0)
            np.fill_diagonal(rh_benefits_nu_ls, 0)
            
            np.fill_diagonal(auto_benefits_eu_ls, 0)
            np.fill_diagonal(trn_benefits_eu_ls, 0)
            np.fill_diagonal(nm_benefits_eu_ls, 0)
            np.fill_diagonal(rh_benefits_eu_ls, 0)
            
            np.fill_diagonal(auto_benefits_nu_cls, 0)
            np.fill_diagonal(trn_benefits_nu_cls, 0)
            np.fill_diagonal(nm_benefits_nu_cls, 0)
            np.fill_diagonal(rh_benefits_nu_cls, 0)
            
            np.fill_diagonal(auto_benefits_eu_cls, 0)
            np.fill_diagonal(trn_benefits_eu_cls, 0)
            np.fill_diagonal(nm_benefits_eu_cls, 0)
            np.fill_diagonal(rh_benefits_eu_cls, 0)
            """
                        
            
            # Calculate the sums
            auto_sum_ls = np.sum(auto_benefits_ls)
            trn_sum_ls = np.sum(trn_benefits_ls)
            nm_sum_ls = np.sum(nm_benefits_ls)
            rh_sum_ls = np.sum(rh_benefits_ls)
            
            # Calculate the sums
            auto_sum_cls = np.sum(auto_benefits_cls)
            trn_sum_cls = np.sum(trn_benefits_cls)
            nm_sum_cls = np.sum(nm_benefits_cls)
            rh_sum_cls = np.sum(rh_benefits_cls)
            
            # Calculate the sums
            auto_sum_nu_ls = np.sum(auto_benefits_nu_ls)
            trn_sum_nu_ls= np.sum(trn_benefits_nu_ls)
            nm_sum_nu_ls = np.sum(nm_benefits_nu_ls)
            rh_sum_nu_ls = np.sum(rh_benefits_nu_ls)
            
            # Calculate the sums
            auto_sum_eu_ls = np.sum(auto_benefits_eu_ls)
            trn_sum_eu_ls = np.sum(trn_benefits_eu_ls)
            nm_sum_eu_ls = np.sum(nm_benefits_eu_ls)
            rh_sum_eu_ls = np.sum(rh_benefits_eu_ls)
            
            # Calculate the sums
            auto_sum_nu_cls = np.sum(auto_benefits_nu_cls)
            trn_sum_nu_cls = np.sum(trn_benefits_nu_cls)
            nm_sum_nu_cls = np.sum(nm_benefits_nu_cls)
            rh_sum_nu_cls = np.sum(rh_benefits_nu_cls)
            
            # Calculate the sums
            auto_sum_eu_cls = np.sum(auto_benefits_eu_cls)
            trn_sum_eu_cls = np.sum(trn_benefits_eu_cls)
            nm_sum_eu_cls = np.sum(nm_benefits_eu_cls)
            rh_sum_eu_cls = np.sum(rh_benefits_eu_cls)
            

            # Append the data to the list
            data_excintz_ls.append([purp, tp_name, auto_suff, auto_sum_ls, trn_sum_ls, nm_sum_ls, rh_sum_ls])
            data_excintz_nu_ls.append([purp, tp_name, auto_suff, auto_sum_nu_ls, trn_sum_nu_ls, nm_sum_nu_ls, rh_sum_nu_ls])
            data_excintz_eu_ls.append([purp, tp_name, auto_suff, auto_sum_eu_ls, trn_sum_eu_ls, nm_sum_eu_ls, rh_sum_eu_ls])
            data_excintz_cls.append([purp, tp_name, auto_suff, auto_sum_cls, trn_sum_cls, nm_sum_cls, rh_sum_cls])
            data_excintz_nu_cls.append([purp, tp_name, auto_suff, auto_sum_nu_cls, trn_sum_nu_cls, nm_sum_nu_cls, rh_sum_nu_cls])
            data_excintz_eu_cls.append([purp, tp_name, auto_suff, auto_sum_eu_cls, trn_sum_eu_cls, nm_sum_eu_cls, rh_sum_eu_cls])
           
                 
# Create a DataFrame from the list
exc_intz_ls = pd.DataFrame(data_excintz_ls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_nu_ls = pd.DataFrame(data_excintz_nu_ls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_eu_ls = pd.DataFrame(data_excintz_eu_ls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_cls = pd.DataFrame(data_excintz_cls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_nu_cls = pd.DataFrame(data_excintz_nu_cls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])
exc_intz_eu_cls = pd.DataFrame(data_excintz_eu_cls, columns=['Purpose', 'Time_Period', 'Auto_Suff','Auto_Benefits', 'Trn_Benefits', 'Nm_Benefits', 'Rh_Benefits'])

# Save the DataFrame to a CSV file
exc_intz_ls.to_csv(_join(benefits_folder,'benefits_data_incintz_bivt_ls.csv'), index=False)
exc_intz_nu_ls.to_csv(_join(benefits_folder,'benefits_data_incintz_bivt_nu_ls.csv'), index=False)
exc_intz_eu_ls.to_csv(_join(benefits_folder,'benefits_data_incintz_bivt_eu_ls.csv'), index=False)
exc_intz_cls.to_csv(_join(benefits_folder,'benefits_data_incintz_bivt_cls.csv'), index=False)
exc_intz_nu_cls.to_csv(_join(benefits_folder,'benefits_data_incintz_bivt_nu_cls.csv'), index=False)
exc_intz_eu_cls.to_csv(_join(benefits_folder,'benefits_data_incintz_bivt_eu_cls.csv'), index=False)